In [1]:
cd ..

C:\Users\14102\Brown\DATA1030\QIM\Data1030_Project


In [2]:
import rioxarray as rxr
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import geopandas as gpd
from scipy.spatial import cKDTree
from scipy.interpolate import interp2d, NearestNDInterpolator, RBFInterpolator
from shapely.geometry import Point
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

In [3]:
# Function from stackoverflow: https://gis.stackexchange.com/questions/222315/finding-nearest-point-in-other-geodataframe-using-geopandas
# Edited the funtion to allow for >1 nearest points to be returned
def ckdnearest(gdA, gdB, num_pts):

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=num_pts)
    gdf = gpd.GeoDataFrame()#gdA.reset_index(drop=True).copy()
    
    for rank, packed_data in enumerate(zip(np.hsplit(idx, num_pts), np.hsplit(dist, num_pts))):
        ids = packed_data[0].flatten()
        dist = packed_data[1].flatten()
        gdB_nearest = gdB.iloc[ids].drop(columns="geometry").reset_index(drop=True)
        
        gdf_t = pd.concat(
            [
                pd.Series(np.full_like(dist, rank + 1), name='distance_rank'),
                gdA.reset_index(drop=True),
                gdB_nearest,
                pd.Series(dist, name='dist')
            ], 
            axis=1)
        
        gdf = gdf.append(gdf_t)

    return gdf


# Population

### Import Population Density data

In [4]:
pop_file = 'Data\Gridded_Population_Estimate_2015_1KM.tif'
pop_2015_tif = rxr.open_rasterio(pop_file)

pop_2015_df = pop_2015_tif[0].to_pandas()

lat_long_dens_df = pop_2015_df.melt(ignore_index=False, value_name='density').reset_index()

lat_long_dens_df = lat_long_dens_df[lat_long_dens_df['density'] > 0]
lat_long_dens_gdf = gpd.GeoDataFrame(
    lat_long_dens_df, geometry=gpd.points_from_xy(lat_long_dens_df.x, 
                                                  lat_long_dens_df.y))
lat_long_dens_gdf.reset_index(drop=True, inplace=True)

# Soil Density

### Import Soil Density Data

In [5]:

# pop_quake_mrt_landslide_file = "Data\GeoSpatial_Data_Processing\Pop_Quake_Mort_Landslide_Soil_Table_Table.xlsx"
# soil_dnsty_df = pd.read_excel(pop_quake_mrt_landslide_file, sheet_name='Pop_Quake_Mort_Landslide_Soil_T')
# soil_dnsty_df.fillna(0, inplace=True)


In [6]:
# soil_dnsty_gdf = gpd.GeoDataFrame(
#     soil_dnsty_df, geometry=gpd.points_from_xy(soil_dnsty_df.POINT_X, 
#                                                soil_dnsty_df.POINT_Y))

# soil_dnsty_gdf.rename({'Density': 'SoilDensity'}, inplace=True)

### Build Soil Density Interpolation Function

In [7]:
# filtered_df = soil_dnsty_gdf#[soil_dnsty_gdf['SoilDensity'] > 0]

# soil_denstiy_intrp = RBFInterpolator(y=filtered_df[['POINT_X', 'POINT_Y']], d=filtered_df['SoilDensity'], kernel='gaussian',
#                                      epsilon=2.5, neighbors=100)

## Predict Soil Density for Each Population Point

In [8]:
# soil_ests = soil_denstiy_intrp(lat_long_dens_gdf[['x', 'y']])

# lat_long_dens_gdf['SoilEst'] = soil_ests

# EarthQuakes

### Import Earthquake data

In [9]:
quake_data_df = pd.read_csv('Data\Worldwide-Earthquake-database.csv')
quake_data_df.set_index('I_D', inplace=True)
quake_data_df.fillna(0, inplace=True)
# quake_data_df['EQ_PRIMARY'] = quake_data_df.apply(lambda x: np.max([x['EQ_PRIMARY'],
#                                                                    x['EQ_MAG_MW'],
#                                                                    x['EQ_MAG_MS'],
#                                                                    x['EQ_MAG_MB'],
#                                                                    x['EQ_MAG_ML'],
#                                                                    x['EQ_MAG_MFA'],
#                                                                    x['EQ_MAG_UNK']]), axis=1)
quake_data_df = quake_data_df.iloc[:, [0,1,2,7,8,9,10,11,12,13,14,15,16,19,20,21,22]]
quake_data_df.reset_index(inplace=True)
quake_data_df_modern = quake_data_df.copy()
quake_data_df_modern = quake_data_df_modern[quake_data_df_modern['YEAR'] >= 1980]
# quake_data_df_modern = quake_data_df_modern[(quake_data_df_modern['EQ_PRIMARY'] > 3)]

quake_data_df_modern = quake_data_df_modern.astype({'LATITUDE': 'float', 'LONGITUDE': 'float'})
quake_data_df_modern.fillna(0, inplace=True)

In [10]:
# Format as GeoDataFrame
quake_data_gdf = gpd.GeoDataFrame(
    quake_data_df_modern, geometry=gpd.points_from_xy(quake_data_df_modern.LONGITUDE, quake_data_df_modern.LATITUDE))

# HDI

### Import HDI (Human Development Index) data

In [11]:
# Import Gridded HDI data / agg
hdi_tif = rxr.open_rasterio('Data/2015_HDI_Data.tif')

hdi_df = hdi_tif[0].to_pandas()

hdi_df = hdi_df.melt(ignore_index=False, value_name='HDI').reset_index()
hdi_df = hdi_df[hdi_df['HDI'] >= 0]
hdi_df[['x', 'y']] = hdi_df[['x', 'y']].round(2)
hdi_df_agg = hdi_df.groupby(['x', 'y']).agg({'HDI': 'mean'}).reset_index()

hdi_gdf = gpd.GeoDataFrame(
    hdi_df_agg, geometry=gpd.points_from_xy(hdi_df_agg.x, hdi_df_agg.y))

### Build HDI Interpolation Function

In [12]:
hdi_fltr = hdi_gdf

hdi_intrp = NearestNDInterpolator(x=hdi_fltr[['x', 'y']], y=hdi_fltr['HDI'])

### Predict HDI for Each Population Point

In [13]:
hdi_ests = hdi_intrp(lat_long_dens_gdf[['x', 'y']])

lat_long_dens_gdf['HDIEst'] = hdi_ests

# Nearest Neighbor Blending

### Merge Earthquake and Geo-Interpolated data by proximity

In [123]:
num_closest_pts = 100
# One degree dif = 60 miles on equator

local_density_dfr_raw = ckdnearest(quake_data_gdf, lat_long_dens_gdf, num_pts=num_closest_pts)
local_density_df = local_density_dfr_raw[((local_density_dfr_raw['dist'] <= 0.5) & (local_density_dfr_raw['EQ_PRIMARY'] <= 4.5)) | 
                        ((local_density_dfr_raw['dist'] <= 1.0) & 
                                    (local_density_dfr_raw['EQ_PRIMARY'] > 4.5) & 
                                    (local_density_dfr_raw['EQ_PRIMARY'] <= 5.5)) |
                                   ((local_density_dfr_raw['dist'] <= 2) & 
                                    (local_density_dfr_raw['EQ_PRIMARY'] > 5.5) &
                                    (local_density_dfr_raw['EQ_PRIMARY'] <= 6.5)) |
                                   ((local_density_dfr_raw['dist'] <= 3.5) & 
                                    (local_density_dfr_raw['EQ_PRIMARY'] > 6.5) &
                                    (local_density_dfr_raw['EQ_PRIMARY'] <= 7.5)) |
                                   ((local_density_dfr_raw['EQ_PRIMARY'] > 7.5))]
local_density_df = local_density_df.reset_index(drop=True)

C:\Users\14102\AppData\Local\Temp\ipykernel_12352\242703894.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(gdf_t)
C:\Users\14102\AppData\Local\Temp\ipykernel_12352\242703894.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(gdf_t)
C:\Users\14102\AppData\Local\Temp\ipykernel_12352\242703894.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(gdf_t)
C:\Users\14102\AppData\Local\Temp\ipykernel_12352\242703894.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(gdf_t)
C:\Users\14102\AppData\Local\Temp\ipykernel_12352\242703894.py:25: FutureWarning: The frame.appe

C:\Users\14102\AppData\Local\Temp\ipykernel_12352\242703894.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(gdf_t)
C:\Users\14102\AppData\Local\Temp\ipykernel_12352\242703894.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(gdf_t)
C:\Users\14102\AppData\Local\Temp\ipykernel_12352\242703894.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(gdf_t)
C:\Users\14102\AppData\Local\Temp\ipykernel_12352\242703894.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(gdf_t)
C:\Users\14102\AppData\Local\Temp\ipykernel_12352\242703894.py:25: FutureWarning: The frame.appe

C:\Users\14102\AppData\Local\Temp\ipykernel_12352\242703894.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(gdf_t)
C:\Users\14102\AppData\Local\Temp\ipykernel_12352\242703894.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(gdf_t)
C:\Users\14102\AppData\Local\Temp\ipykernel_12352\242703894.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(gdf_t)
C:\Users\14102\AppData\Local\Temp\ipykernel_12352\242703894.py:25: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  gdf = gdf.append(gdf_t)
C:\Users\14102\AppData\Local\Temp\ipykernel_12352\242703894.py:25: FutureWarning: The frame.appe

In [116]:
# Aggregate over n closest points

wm = lambda x: np.average(x, weights=local_density_df.loc[x.index, 'density'])
d_wgt = lambda x: np.average(x, weights=local_density_df.loc[x.index, 'dist'])

local_density_agg = local_density_df.groupby(list(quake_data_df_modern.columns)[:-1]).agg(avg_density=('density', 'mean'), 
                                                                                          dst_avg_density=('density', d_wgt), 
                                                                                          sum_density=('density', 'sum'),
                                                                                          density_deviation=('density', 'std'),
                                                                                          max_density=('density', 'max'),
                                                                                          HDI=('HDIEst', 'mean'),
                                                                                          HDI_pop_wgt=('HDIEst',  wm),
                                                                                          max_HDI=('HDIEst',  'max'),
                                                                                          dist_to_closest_pop=('dist', 'min'),
                                                                                          pop_dist_deviation=('dist', 'std'),
                                                                                          avg_pop_distance=('dist', 'mean'))
local_density_agg.reset_index(inplace=True)
local_density_agg = gpd.GeoDataFrame(local_density_agg)
local_density_agg['eq_mag_unlogged'] = 10**local_density_agg['EQ_PRIMARY']

### Import Gridded Land Coverage Data

In [17]:
# Import Gridded HDI data / agg
# Experimental. Not using this data at the moment.
# land_use_tif = rxr.open_rasterio("Data/Gridded_Land_Cover_Categories.tif")  # Change this back to land use data

# land_use_df = land_use_tif[0].to_pandas()
# land_use_df = land_use_df.iloc[::100, ::100]  # Way too much data, need to trim down
# land_use_df = land_use_df.melt(ignore_index=False, value_name='LandUseCode').reset_index()

In [18]:
# land_use_df[['x', 'y']] = land_use_df[['x', 'y']].round(2)
# land_use_agg = land_use_df.groupby(['x', 'y', 'LandUseCode']).size().reset_index().iloc[:,:-1]
# land_use_gdf = gpd.GeoDataFrame(
#     land_use_agg, geometry=gpd.points_from_xy(land_use_agg.x, land_use_agg.y))

### Merge Earthquake and Land Cover data by proximity

In [19]:
# local_density_hdi_landcvr_df = ckdnearest(local_hdi_agg, land_use_gdf, num_pts=1)


In [20]:
# local_density_hdi_landcvr_df.drop(['x', 'y', 'distance_rank', 'geometry'], axis=1, inplace=True)
# local_density_hdi_landcvr_df.rename({'dist': 'dist_to_land_code'}, inplace=True, axis=1)

### Ouput data as csv

In [118]:
local_density_agg = local_density_agg[local_density_agg['EQ_PRIMARY'] > 3].copy()
local_density_agg.fillna(0, inplace=True)
local_density_agg.to_csv('Data/blended_quake_data.csv')

In [124]:
local_density_df = local_density_df[local_density_df['EQ_PRIMARY'] > 3].copy().fillna(0)

In [125]:
local_density_df.to_csv('Data/quake_data_no_agg.csv')